In [ ]:
"""This program trains a binary classification model on image data (human face images with and without eyeglasses), 
saves the trained model, and classifies new/test images using the model. Also called binary image classification model. 
It constructs a custom model on top of the pre-trained base model employing transfer learning.
The trained model predicts/classifies the test/new images as: 0 - No Eye glasses & 1 - Eye glasses present.

This program will be hosted online via Streamlit community cloud. 
Streamlit web app will be created that allows users to upload an image, classify it using pre-trained models, and
display the result."""

In [ ]:
# Importing libraries
import os
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Define training and validation directories
train_data_dir = 'train'
validation_data_dir = 'test'

# Image dimensions
img_width, img_height = 224, 224

# Setting up training parameters
epochs = 10
batch_size = 100

# Create base model with pre-trained weights on ImageNet
base_model = MobileNetV2(input_shape=(img_width, img_height, 3), include_top=False, weights='imagenet')

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Build your own model on top of the base model
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Normalization for validation/testing
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,  # You can adjust the number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Save the trained model
model.save('Model4.h5')

In [ ]:
# Generating Classification report

import os
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Define training and validation directories
train_data_dir = 'train'
validation_data_dir = 'test'

# Image dimensions
img_width, img_height = 224, 224

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Normalization for validation/testing
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

from sklearn.metrics import classification_report

# Load the saved model
from keras.models import load_model
model = load_model('Model4.h5')

# Generate predictions for the validation dataset
validation_generator.reset()  # Reset generator to start from beginning
y_pred = model.predict_generator(validation_generator, steps=len(validation_generator), verbose=1)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

# Get true labels
y_true = validation_generator.classes

# Generate classification report
print(classification_report(y_true, y_pred_binary))

In [ ]:
# Plotting loss and accuracy over epochs
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(20,5))

# Plotting loss & validation loss
plt.subplot(1,2,1)
sns.lineplot(x=history.epoch, y=history.history['loss'], color='red', label='Train Loss')
sns.lineplot(x=history.epoch, y=history.history['val_loss'], color='orange', label='Val Loss')
plt.title('Loss on train vs test')
plt.legend(loc='best')

# Plotting accuracy and validation accuracy
plt.subplot(1,2,2)
sns.lineplot(x=history.epoch, y=history.history['accuracy'], color='blue', label='Train Accuracy')
sns.lineplot(x=history.epoch, y=history.history['val_accuracy'], color='green', label='Val Accuracy')
plt.title('Accuracy on train vs test')
plt.legend(loc='best')

plt.show()

In [ ]:
# Loading the trained model, using it to classify for images in batch
# and exporting classification results to an Excel file.

from keras.models import load_model
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np
import os

model = load_model('Model4.h5')

# Making predictions on test/new images using the trained model from the above.
import glob
import pandas as pd

# Setting the directory for testing images
folder_dir = "/Users/tabal/Downloads/new_faces"

# Create an empty list to store prediction results
prediction_results = []

# Looping through each image in the directory
for image in glob.iglob(f'{folder_dir}/*'):
    
    # Loading and preprocessing the image
    load_image = load_img(image, target_size=(224, 224))
    img = img_to_array(load_image)
    img = preprocess_input(img.reshape(1,224,224,3))
    
    # Making predictions using the loaded model
    label = model.predict(img)
    
    # Append the prediction results to the list
    prediction_results.append({
        'Image Name': os.path.basename(image),
        'Prediction': round(label[0][0])})

# Create a DataFrame from the list
df = pd.DataFrame(prediction_results)

# Save the DataFrame to an Excel file
df.to_excel('classification_results_Model4.xlsx', index=False)